# Adaboost classifier class

In [ ]:
# Import guys
import numpy as np

In [1]:
# Class

class AdaBoostClassifer:
    """
    Implementation of adaboost - with custom loss function
    """
    def __init__(self, n_estimators, lr, loss_fn=None):
        """
        Args:
            n_estimators = number of stumps the final tree will be built from
            lr = learning rate
        """
        self.n_estimators = n_estimators
        self.lr = lr
        self.stumps = []
        self.weights = None

    def exp_loss(self, y_true, y_pred):
        loss = 

    

SyntaxError: incomplete input (655450674.py, line 9)

: 